In [13]:
import cv2
import numpy as np
import pytesseract
import spacy
from spellchecker import SpellChecker
import json
from spacy.pipeline import EntityRuler


In [4]:

def preprocess_text_image(image_file):
    # Load the input image
    image = cv2.imread(image_file)
    
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Binarization
    thresh, im_bw = cv2.threshold(gray_image, 210, 230, cv2.THRESH_BINARY)
    
    # Noise Removal
    def noise_removal(image):
        kernel = np.ones((1, 1), np.uint8)
        image = cv2.dilate(image, kernel, iterations=1)  # Dilate the image to remove noise 
        kernel = np.ones((1, 1), np.uint8)
        image = cv2.erode(image, kernel, iterations=1)
        image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
        image = cv2.medianBlur(image, 3)
        return image

    no_noise = noise_removal(im_bw)

    # Perform text extraction using Tesseract OCR
    extracted_text = pytesseract.image_to_string(no_noise)


    # Save the preprocessed image
    cv2.imwrite("preprocessed_image.jpg", no_noise)

    # Save the extracted text to a text file
    with open("extracted_text.txt", "w") as text_file:
        text_file.write(extracted_text)
    
    return extracted_text


In [6]:
# Example usage
image_file = '01.jpg'
extracted_text= preprocess_text_image(image_file)
print(extracted_text)


Lettre de Motivation Simple

Pherre Letetrere
2A rue du Cet
45000 Oridans
06 06 06 06 06
pierre lafebvre@pnad.com

[NOM DE LENTREPRISE;
[ADRESSE DL VEN TREPRISE;

A{UEU), le (OATE)
Objet : Concidature ou poste de [nom du poste suquel tu pottules]

Madame, Monseut,

Par % buns te La presente e€ du CV c-jomnt, je Lounaite airs soumatte mon profil et
Yous expamer toute ma motryaton pour exercer les fonct-ons de [empha pour ‘quel tu
postules - réf | av semn de votre service de [sence pow kequei ty postules])

 

Lots de mes derneres expénences protessonne'les, fa travalié pour difterentes
entropies du secteur jsecteur ov secteurs dans Iesquets tu as trawalé), Co fai pu acquére
#t céveloppar de sobdes connassances en [mantionne les programmes ou .e comanes,
dans lesquets tu a3 acquis des connassances]. entre autres, que yaimerars partager avec
vous et continurr & approfondy au se:n Tune équipe dyuamaue et suinutante.

 

‘Au cours de ma cerdre expérence professionnelle a/cher (Gernuere entr

In [15]:
def process_text(text):
    # Load the spaCy French model
    nlp = spacy.load("fr_core_news_sm")
    
    # Define custom patterns for named entities
    custom_patterns = [
        {"label": "PERSON", "pattern": [{"LOWER": {"IN": ["mr.", "mrs.", "ms.", "dr.", "prof."]}}, {"IS_ALPHA": True, "IS_TITLE": True}]},
        {"label": "ORG", "pattern": [{"TEXT": {"REGEX": "^(inc|llc|ltd|corp|s\\.?a\\.?|s\\.p\\.a|s\\.r\\.l|s\\.c\\.a)$"}}]}
        # Add more custom patterns for other entities as needed
    ]
    
    # Add custom patterns to the entity ruler
    ruler = EntityRuler(nlp)
    for pattern in custom_patterns:
        ruler.add_patterns([pattern])
    nlp.add_pipe(ruler)

    # Analyze the text using spaCy
    doc = nlp(text)
    
    # Extract named entities
    named_entities = [{'text': ent.text, 'label': ent.label_} for ent in doc.ents]
    
    # Perform spell checking for French text
    spell = SpellChecker(language='fr')
    corrected_text_tokens = []
    for word in text.split():
        correction = spell.correction(word)
        corrected_text_tokens.append(correction if correction is not None else word)
    corrected_text = ' '.join(corrected_text_tokens)
    
    # Structure the information into JSON
    output = {
        'original_text': text,
        'corrected_text': corrected_text,
        'named_entities': named_entities
    }
    
    return output


In [16]:
processed_data = process_text(extracted_text)

# Save the processed data to a JSON file
with open('processed_data.json', 'w') as json_file:
    json.dump(processed_data, json_file, indent=4)

print("Processed data saved to processed_data.json")

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <spacy.pipeline.entityruler.EntityRuler object at 0x75ad6700f580> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.